In [8]:
import wget                     
from Bio import SeqIO
import sys

## Objective 

* Extract a subsequence from a given primary multi-fasta file (https://ftp.ncbi.nlm.nih.gov/genomes/Viruses/Vieuvirus.fn) and a bed file. The extracted subsequence must be removed from the primary sequence (multifasta). You can create your own BED file. It should contain coordinates for at least 3 scaffolds.
* Reverse complement the extracted sequences.
* Introduce random 1 nucleotide change in the above extracted sequence(s); this becomes the processed subsequence(s).
* Insert the processed subsequence(s) back to the primary sequence(multifasta). 




## Download the multifasta file

In [9]:
file = wget.download('https://ftp.ncbi.nlm.nih.gov/genomes/Viruses/Vieuvirus.fn')
file

'Vieuvirus.fn'

## Designing BED file for whole multifasta file

In [10]:

with open('Vieuvirus.fn') as multifasta_file, open('sequences.bed','w') as output_bed_file:
    for record in SeqIO.parse(multifasta_file, 'fasta'):
        output_bed_file.write('{}\t0\t{}\n'.format(record.id, len(record)))

In [11]:

with open('sequences.bed') as my_file:
    output = my_file.read()
print(output)


gi|401824227|gb|JX403940.1|_Acinetobacter_phage_YMC/09/02/B1251_ABA_BP,_complete_genome	0	45364
gi|768216015|gb|KP861230.1|_Acinetobacter_phage_YMC11/11/R3177,_complete_genome	0	47575
gi|940328418|gb|_KT588073_Acinetobacter_phage_Ab105-3phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-3phi]_.1|_Acinetobacter_phage_Ab105-3phi,_partial_genome	0	63785
gi|1215353646|gb|_KT588075_Acinetobacter_phage_Ab105-2phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-2phi]_.2|_Acinetobacter_phage_Ab105-2phi,_complete_genome	0	61304
gi|1043841987|gb|_KX268652_Acinetobacter_phage_vB_AbaS_TRS1__[-]_[Vieuvirus]_[Acinetobacter_phage_vB_AbaS_TRS1]_.1|_Acinetobacter_phage_vB_AbaS_TRS1,_complete_genome	0	40749
gi|1384047875|gb|_MH115576_Acinetobacter_phage_AM106__[-]_[Vieuvirus]_[Acinetobacter_phage_AM106]_.1|_Acinetobacter_phage_AM106,_complete_genome	0	52031
gi|1841190245|gb|_MT361972_Acinetobacter_phage_Ab11510-phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab11510-phi]_.1|_Acinetobacter_phage_Ab11510-phi,_complete

In [12]:
with open("sequences.bed") as my_file:
    lines = len(my_file.readlines())

print('Total number of lines (corresponding sequences) in bed file is: ', lines)

Total number of lines (corresponding sequences) in bed file is:  22


In [13]:
# Let's look at all the fasta entries it has
with open('Vieuvirus.fn') as my_file:
    output_fasta = my_file.read()
print(output_fasta)

>gi|401824227|gb|JX403940.1|_Acinetobacter_phage_YMC/09/02/B1251_ABA_BP,_complete_genome
...
CTAA
>gi|768216015|gb|KP861230.1|_Acinetobacter_phage_YMC11/11/R3177,_complete_genome
...
>gi|940328418|gb|_KT588073_Acinetobacter_phage_Ab105-3phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-3phi]_.1|_Acinetobacter_phage_Ab105-3phi,_partial_genome
...
ATTTATAACAAAAAT
>gi|1215353646|gb|_KT588075_Acinetobacter_phage_Ab105-2phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-2phi]_.2|_Acinetobacter_phage_Ab105-2phi,_complete_genome
...
>gi|1043841987|gb|_KX268652_Acinetobacter_phage_vB_AbaS_TRS1__[-]_[Vieuvirus]_[Acinetobacter_phage_vB_AbaS_TRS1]_.1|_Acinetobacter_phage_vB_AbaS_TRS1,_complete_genome
...
GGGGTGGTT
>gi|1384047875|gb|_MH115576_Acinetobacter_phage_AM106__[-]_[Vieuvirus]_[Acinetobacter_phage_AM106]_.1|_Acinetobacter_phage_AM106,_complete_genome
...
TTGGTTTAAAATTCTATTGGA
>gi|1841190245|gb|_MT361972_Acinetobacter_phage_Ab11510-phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab11510-phi]_.1|_Acine

'''Now we will convert the output fasta string containing all the sequences to a list, 
to help accessing the sequence that we want to process'''

In [14]:
sequence_list = output_fasta.split(">")
print(len(sequence_list))  # ['','>gi ....', '>gi .....' , '>gi ....']

23


## Now, let's take first fasta file to complete rest of objectives

In [15]:
sequence_list[1]

'gi|401824227|gb|JX403940.1|_Acinetobacter_phage_YMC/09/02/B1251_ABA_BP,_complete_genome\...\nCTAA\n'

In [16]:
# Above after splitting and conversion to list we lost '>' character, therefor adding it
extracted_seq = '>'+ sequence_list[1]
extracted_seq

'>gi|401824227|gb|JX403940.1|_Acinetobacter_phage_YMC/09/02/B1251_ABA_BP,_complete_genome\...\nCTAA\n'

In [17]:
# Save the working sequence in a new file
with open('extracted_seq.fasta','w') as outfile:
    outfile.write(extracted_seq)

In [18]:
# Let's look at specifics of the sequence under study
with open('extracted_seq.fasta') as file:
    for seq_record in SeqIO.parse(file, "fasta"):
        record_id = seq_record.id
        # print(seq_record.name)
        print(seq_record.id)
        print(repr(seq_record.seq))
        removal_seq = seq_record.seq
        print('length of sequence:', len(seq_record))

gi|401824227|gb|JX403940.1|_Acinetobacter_phage_YMC/09/02/B1251_ABA_BP,_complete_genome
Seq('TAAAATATTTATGTTTTTATGGTCGTCATGATTGGTGAAATCATTCCAAGGTTT...TAA')
length of sequence: 45364


## Remove the sequence and append to a new file

In [19]:
# Parse the parent file and append the rest seq.id's to a new file
with open('edited.fn','w') as multifasta_file, open('extracted_seq.fasta','r') as output_file:
    remove = output_file.read() 
    
    for seq in SeqIO.parse('Vieuvirus.fn', 'fasta'):
        if seq.id not in remove:
            SeqIO.write(seq, multifasta_file, "fasta")

In [20]:

count = 0
for seq_record in SeqIO.parse('edited.fn', "fasta"):
    count+=1
    print(seq_record.id)
    # print(repr(seq_record.seq))
    # print(len(seq_record))

print('\nCount of sequences after removal:', count) # count is 21, thus we removed one

gi|768216015|gb|KP861230.1|_Acinetobacter_phage_YMC11/11/R3177,_complete_genome
gi|940328418|gb|_KT588073_Acinetobacter_phage_Ab105-3phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-3phi]_.1|_Acinetobacter_phage_Ab105-3phi,_partial_genome
gi|1215353646|gb|_KT588075_Acinetobacter_phage_Ab105-2phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-2phi]_.2|_Acinetobacter_phage_Ab105-2phi,_complete_genome
gi|1043841987|gb|_KX268652_Acinetobacter_phage_vB_AbaS_TRS1__[-]_[Vieuvirus]_[Acinetobacter_phage_vB_AbaS_TRS1]_.1|_Acinetobacter_phage_vB_AbaS_TRS1,_complete_genome
gi|1384047875|gb|_MH115576_Acinetobacter_phage_AM106__[-]_[Vieuvirus]_[Acinetobacter_phage_AM106]_.1|_Acinetobacter_phage_AM106,_complete_genome
gi|1841190245|gb|_MT361972_Acinetobacter_phage_Ab11510-phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab11510-phi]_.1|_Acinetobacter_phage_Ab11510-phi,_complete_genome
gi|2085573605|gb|_MZ514874_Acinetobacter_phage_Ab105-2phideltaCI404ad__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-2phideltaCI404

In [21]:
type(removal_seq) # is seq object, we will convert it into str object for feasibility

Bio.Seq.Seq

In [22]:
removal_seq = str(removal_seq)
type(removal_seq)

str

## Reverse Complement

In [23]:
# Function for reverse complement
def find_reverse_complement(dna_sequence):
    complement_map = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    reverse_complement_sequence = ''.join(complement_map[base] for base in dna_sequence[::-1])
    return reverse_complement_sequence

In [24]:
reverse = find_reverse_complement(removal_seq)
print(reverse)

...


## Mutating the sequence at a random location

In [25]:
# First lets convert the str object to list, for ease of processing
reverse = list(reverse)
reverse[2] = "C"
reverse_mutated = ''.join(base for base in reverse)
reverse_mutated

'...'

## Convert the reverse complement to a Seq object 

In [26]:
from Bio.Seq import Seq
new_seq = Seq(reverse_mutated)
type(new_seq) # seq object

Bio.Seq.Seq

In [27]:
# Convert the sequence into seq record and append toa new file
from Bio.SeqRecord import SeqRecord

record = SeqRecord(new_seq, "gi|401824227|gb|JX403940.1|_Acinetobacter_phage_YMC/09/02/B1251_ABA_BP,_complete_genome")
SeqIO.write(record, "mutated_seq.fasta", "fasta")

1

## Insert the mutated sequence with others

In [28]:
from Bio import SeqIO
count = 0
for seq_record in SeqIO.parse('edited.fn', "fasta"):
    count+=1
    print(seq_record.id)

print('\nCount of sequences before addition of mutated sequence:', count)

gi|768216015|gb|KP861230.1|_Acinetobacter_phage_YMC11/11/R3177,_complete_genome
gi|940328418|gb|_KT588073_Acinetobacter_phage_Ab105-3phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-3phi]_.1|_Acinetobacter_phage_Ab105-3phi,_partial_genome
gi|1215353646|gb|_KT588075_Acinetobacter_phage_Ab105-2phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-2phi]_.2|_Acinetobacter_phage_Ab105-2phi,_complete_genome
gi|1043841987|gb|_KX268652_Acinetobacter_phage_vB_AbaS_TRS1__[-]_[Vieuvirus]_[Acinetobacter_phage_vB_AbaS_TRS1]_.1|_Acinetobacter_phage_vB_AbaS_TRS1,_complete_genome
gi|1384047875|gb|_MH115576_Acinetobacter_phage_AM106__[-]_[Vieuvirus]_[Acinetobacter_phage_AM106]_.1|_Acinetobacter_phage_AM106,_complete_genome
gi|1841190245|gb|_MT361972_Acinetobacter_phage_Ab11510-phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab11510-phi]_.1|_Acinetobacter_phage_Ab11510-phi,_complete_genome
gi|2085573605|gb|_MZ514874_Acinetobacter_phage_Ab105-2phideltaCI404ad__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-2phideltaCI404

In [29]:
with open('edited.fn', "a") as f, open('mutated_seq.fasta', "r") as r:
    # add = f.read()
    for seq in SeqIO.parse(r, 'fasta'): 
        # if seq.id not in add:
            SeqIO.write(seq, f, "fasta")

In [30]:
from Bio import SeqIO
count = 0
for seq_record in SeqIO.parse('edited.fn', "fasta"):
    count+=1
    print(seq_record.id)
    # print(repr(seq_record.seq))
    # print(len(seq_record))

print('\nCount of sequences after addition of mutated sequence:', count)

gi|768216015|gb|KP861230.1|_Acinetobacter_phage_YMC11/11/R3177,_complete_genome
gi|940328418|gb|_KT588073_Acinetobacter_phage_Ab105-3phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-3phi]_.1|_Acinetobacter_phage_Ab105-3phi,_partial_genome
gi|1215353646|gb|_KT588075_Acinetobacter_phage_Ab105-2phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-2phi]_.2|_Acinetobacter_phage_Ab105-2phi,_complete_genome
gi|1043841987|gb|_KX268652_Acinetobacter_phage_vB_AbaS_TRS1__[-]_[Vieuvirus]_[Acinetobacter_phage_vB_AbaS_TRS1]_.1|_Acinetobacter_phage_vB_AbaS_TRS1,_complete_genome
gi|1384047875|gb|_MH115576_Acinetobacter_phage_AM106__[-]_[Vieuvirus]_[Acinetobacter_phage_AM106]_.1|_Acinetobacter_phage_AM106,_complete_genome
gi|1841190245|gb|_MT361972_Acinetobacter_phage_Ab11510-phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab11510-phi]_.1|_Acinetobacter_phage_Ab11510-phi,_complete_genome
gi|2085573605|gb|_MZ514874_Acinetobacter_phage_Ab105-2phideltaCI404ad__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-2phideltaCI404

## Updating the bed file

In [31]:
# Annotate the bed file accordingly as to what we did!
with open('Vieuvirus.fn') as multifasta_file, open('updated_sequences.bed','w') as output_bed_file:
    for seq in SeqIO.parse(multifasta_file, 'fasta'):
        if seq.id == record_id:
            output_bed_file.write('{}\t0\t{}\t{}\n'.format(seq.id, len(seq), 'Mutated sequence'))
        else: 
            output_bed_file.write('{}\t0\t{}\t\n'.format(seq.id, len(seq)))

In [32]:
with open('updated_sequences.bed', 'r') as file:
    result = file.read()
print(result)

gi|401824227|gb|JX403940.1|_Acinetobacter_phage_YMC/09/02/B1251_ABA_BP,_complete_genome	0	45364	Mutated sequence
gi|768216015|gb|KP861230.1|_Acinetobacter_phage_YMC11/11/R3177,_complete_genome	0	47575	
gi|940328418|gb|_KT588073_Acinetobacter_phage_Ab105-3phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-3phi]_.1|_Acinetobacter_phage_Ab105-3phi,_partial_genome	0	63785	
gi|1215353646|gb|_KT588075_Acinetobacter_phage_Ab105-2phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab105-2phi]_.2|_Acinetobacter_phage_Ab105-2phi,_complete_genome	0	61304	
gi|1043841987|gb|_KX268652_Acinetobacter_phage_vB_AbaS_TRS1__[-]_[Vieuvirus]_[Acinetobacter_phage_vB_AbaS_TRS1]_.1|_Acinetobacter_phage_vB_AbaS_TRS1,_complete_genome	0	40749	
gi|1384047875|gb|_MH115576_Acinetobacter_phage_AM106__[-]_[Vieuvirus]_[Acinetobacter_phage_AM106]_.1|_Acinetobacter_phage_AM106,_complete_genome	0	52031	
gi|1841190245|gb|_MT361972_Acinetobacter_phage_Ab11510-phi__[-]_[Vieuvirus]_[Acinetobacter_phage_Ab11510-phi]_.1|_Acinetobacter_phage